In [1]:
import tensorflow as tf
import numpy as np

# Basics and installation: http://www.jorditorres.org/first-contact-with-tensorflow/
# Useful example to print and run:  http://katbailey.github.io/post/matrix-factorization-with-tensorflow/

# Other way to install: 
'''
python --version
conda create -n tensorflow python=2.7
source activate tensorflow
cd ..
mkdir tf
cd tf
export TF_BINARY_URL=https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-0.10.0-py2-none-any.whl
conda install -c conda-forge tensorflow
'''

'\npython --version\nconda create -n tensorflow python=2.7\nsource activate tensorflow\ncd ..\nmkdir tf\ncd tf\nexport TF_BINARY_URL=https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-0.10.0-py2-none-any.whl\nconda install -c conda-forge tensorflow\n'

# Basic values and placeholders

In [2]:
random_scalar = tf.random_uniform(())
sess = tf.Session()
sess.run(random_scalar)

0.62619925

Both TensorFlow and NumPy allow nearly any variable to take the form of a tensor (i.e., a vector, a matrice, or a higher-order such structure). We can do random uniform within a range and put this in float or integer


In [3]:
random_tensor = tf.random_uniform((2,3),minval=0, maxval=10,dtype=tf.int64)
sess = tf.Session()
sess.run(random_tensor)

array([[1, 2, 5],
       [1, 1, 8]])

To create a function, it should return a tensor. And we should call it by a session with feedback: 

In [4]:
def tf_fn(x):
    return tf.mul(x,2)

x = tf.placeholder("float", 4)
func = tf_fn(x)

with tf.Session() as session:
    result = session.run(func, feed_dict={x:  np.array([0, 1, 2, 3])})
    print(result)
    

[ 0.  2.  4.  6.]


In [5]:
a = tf.placeholder("float")
b = tf.placeholder("float")
y = tf.mul(a, b)

sess = tf.Session()
print  sess.run(y, feed_dict={a: 3, b: 3})

9.0


# Matrix multiplication


In [6]:

##### Variables with vectors
dim1 = 2   # columns = rank = features
dim2 = 2   # rows = observations
M1 = tf.Variable(tf.random_uniform([dim1,dim2],minval=0, maxval=10,dtype=tf.int32))
M2 = tf.Variable(tf.random_uniform([dim2,dim1],minval=0, maxval=10,dtype=tf.int32)) 

L1 = tf.Variable( [ [2],[3] ] )               # Two and three times the vector
#L1 = tf.Variable( [ [2] ] )                   # Two times the vector
L2 = tf.Variable( [[1,1,2],[1,2,3]] )         # This is the vector

c = tf.constant(1.0)

##### Operations
M = tf.reduce_sum( tf.mul(M1,M2), 1)
M_M = tf.matmul(M1,tf.reshape(M2,[dim2,dim1]))
L = tf.mul(L1,L2)

##### Tensorflow Session
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

##### Print results

results = sess.run([M1,M2,M,M_M])
print results[0]
print results[1]
print "mul result (one by one): ", results[2], "\n"
print "dot product or matrix mult with matmul result (matrix): ", results[3], "\n"

results = sess.run([L1,L2,L])
print results[0]
print results[1]
print "mul result with defined vector: ", results[2], "\n"

print sess.run(c)


[[2 0]
 [5 3]]
[[9 4]
 [7 6]]
mul result (one by one):  [18 53] 

dot product or matrix mult with matmul result (matrix):  [[18  8]
 [66 38]] 

[[2]
 [3]]
[[1 1 2]
 [1 2 3]]
mul result with defined vector:  [[2 2 4]
 [3 6 9]] 

1.0


# Look for values in a matrix

In [7]:
matrix = [[ 0. ,2. ,1. ,1. ,1., 1. , 1.], [ 2. , 0. , 0., 3. , 1.,  1. , 5.], [ 0. , 0. , 0.,  1. , 1.,  1. , 0.],
          [ 1. , 1.,  1.,  0.,  1.,  1.,  0.], [ 1. , 0. , 1. , 1. , 0. , 1. , 0.], 
          [ 1.,  1.,  1.,  1.,  1.,  0.,  0.], [ 1. , 1.,  0.,  0.,  0. , 0. , 0.]]
matrix


[[0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [2.0, 0.0, 0.0, 3.0, 1.0, 1.0, 5.0],
 [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
 [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0],
 [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [109]:
all_index = set([(1, 3), (3, 0), (5, 4), (1, 6),(1,4),(0,1)])
index_to_look = set([(1, 3), (3, 0), (5, 4), (1, 6)])
index_to_look 

{(1, 3), (1, 6), (3, 0), (5, 4)}

In [124]:

##### Variables in tensorflow
params = tf.Variable( matrix )        
ids = tf.Variable( list(index_to_look)  ) 

ids_position1 = tf.gather_nd(ids,tf.ones(len(ide)))

vectors_selection = tf.gather_nd(params, ids)

##### Tensorflow Session
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

##### Print results
print sess.run( vectors_selection )
print sess.run( ids )
print sess.run( ids_position1 )


[ 1.  1.  3.  5.]
[[3 0]
 [5 4]
 [1 3]
 [1 6]]
[1 1 1 1]


# Global Vectors Algorithm example

In [106]:

##### Variables in tensorflow
params = tf.Variable( matrix )        
ids = tf.Variable( list(index_to_look)  ) 
vectors_selection = tf.gather_nd(params, ids)

dim1 = len(index_to_look)
dim2 = 3
M1 = tf.Variable(tf.random_uniform([dim1,dim2],minval=0, maxval=1,dtype=tf.float32))
M2 = tf.Variable(tf.random_uniform([dim1,dim2],minval=0, maxval=1,dtype=tf.float32))
M = tf.mul(M1,M2)
M_r = tf.reduce_sum(M,1)
b = tf.Variable(tf.ones([dim1]),dtype=tf.float32)  

X = tf.log(vectors_selection)
fmax = tf.minimum(X,tf.ones([dim1]))

weights =  tf.Variable(tf.random_uniform([len(all_index),dim2],minval=0, maxval=1,dtype=tf.float32))
wi = [i for (i,j) in index_to_look]  
wj =  [j for (i,j) in index_to_look]  


result = M_r + b + X
result_sqr = result**2

comparison = tf.ones(dim1)*5.
quares = result_sqr < comparison
        
##### Tensorflow Session
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)


print "weights i per word: ",sess.run(M1)
print "weights j per word: ",sess.run(M2)

print "\n"
print "weights mult ij: ",sess.run(M)
print "weights dot prod result ij: ",sess.run(M_r)
print "\n"
print "X input (coocurrences): ",sess.run(X)
print "W", sess.run(weights)
print "W", wi
print "W", sess.run(weights)[wi,:]
print "results fminimum: ",sess.run(fmax)
print "\n"
print "results costs: ",sess.run(result)
print "results sqr costs: ", sess.run(result_sqr)
print "mean reduction for the minibatch: ",sess.run(tf.reduce_mean(result))

## numpy subselection of matrix by indices

print "NUMPY SELECTION EQUIVALENT TO TF.GATHER", list(zip(*index_to_look)[0])


weights i per word:  [[ 0.32101893  0.00588596  0.17248464]
 [ 0.48212528  0.82793736  0.8897953 ]
 [ 0.74746346  0.24163473  0.10198379]
 [ 0.42474997  0.06667519  0.28107142]]
weights j per word:  [[ 0.32573187  0.16539848  0.74664092]
 [ 0.819417    0.76093197  0.51857352]
 [ 0.07197762  0.53984177  0.63790977]
 [ 0.93306565  0.4713881   0.99553132]]


weights mult ij:  [[ 0.1045661   0.00097353  0.12878409]
 [ 0.39506164  0.63000399  0.46142429]
 [ 0.05380064  0.13044451  0.06505645]
 [ 0.3963196   0.03142989  0.27981541]]
weights dot prod result ij:  [ 0.23432371  1.48649001  0.24930161  0.70756489]


X input (coocurrences):  [ 0.          0.          1.09861231  1.60943794]
W [[ 0.58161545  0.59163451  0.64252329]
 [ 0.50284922  0.88060546  0.45352566]
 [ 0.87048972  0.47148263  0.95153987]
 [ 0.3252964   0.5925988   0.83370912]
 [ 0.65519285  0.17015147  0.00559783]
 [ 0.27518439  0.9471159   0.17665291]]
W [3, 5, 1, 1]
W [[ 0.3252964   0.5925988   0.83370912]
 [ 0.27518439  0.9

# PREMISE AND HYPOTHESIS EXAMPLE CONCATENATED 

In [63]:

import tensorflow as tf 

dim1 = 3
dim2 = 3

M1 = tf.Variable(tf.random_uniform([dim1,dim2],minval=-20, maxval=20))
M2 = tf.Variable(tf.random_uniform([dim2,dim1],minval=-20, maxval=20)) 

W_cl = tf.Variable(tf.ones([4*dim1, 1]))

W_rel = tf.Variable(tf.ones([4*dim1, 1]))
W_relcl = tf.Variable(tf.ones([1, 1]))

b_cl = tf.Variable(tf.ones([1]))

M = tf.concat(1,[M1,M2,M1-M2,M1*M2])

logs = tf.matmul(M,W_cl)+ b_cl
soft = tf.nn.softmax(logs)

relu = tf.nn.relu(tf.matmul(M,W_rel)+ b_cl)
logsrelu = tf.matmul(relu,W_relcl)+ b_cl
softrelu = tf.nn.softmax(logsrelu)

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

##### Print results

results = sess.run([M1,M2,M,W_cl,b_cl,logs,soft,relu,logsrelu,softrelu])
print "Hp\n",results[0]
print "Hh\n",results[1]
print "Concatenated\n",results[2]
print "W\n",results[3].transpose()
print "b\n",results[4]
print "logits\n",results[5]
print "maxsoft\n",results[6]
print "relu\n",results[7]
print "logitssrelu\n",results[8]
print "maxsoftrelu\n",results[9]



Hp
[[ 17.54130554  -8.90909195 -11.56913757]
 [ -6.45233631   6.53700829   1.03490829]
 [ 18.73347473  -2.52202034 -18.44117165]]
Hh
[[  9.42677498 -16.72008896   4.15140533]
 [-19.28181076  -9.3717289   -7.19891548]
 [ -2.93381119  -3.1003418   17.21966553]]
Concatenated
[[  17.54130554   -8.90909195  -11.56913757    9.42677498  -16.72008896
     4.15140533    8.11453056    7.81099701  -15.72054291  165.35794067
   148.96081543  -48.02817917]
 [  -6.45233631    6.53700829    1.03490829  -19.28181076   -9.3717289
    -7.19891548   12.82947445   15.90873718    8.23382378  124.41272736
   -61.26306915   -7.45021725]
 [  18.73347473   -2.52202034  -18.44117165   -2.93381119   -3.1003418
    17.21966553   21.66728592    0.57832146  -35.66083527  -54.96047592
     7.81912518 -317.55081177]]
W
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]
b
[ 1.]
logits
[[ 261.41674805]
 [  58.93860626]
 [-368.15161133]]
maxsoft
[[ 1.]
 [ 1.]
 [ 1.]]
relu
[[ 261.41674805]
 [  58.93860626]
 [   0.      

# PREMISE AND HYPOTHESIS EXAMPLE CONCATENATED 

In [63]:

import tensorflow as tf 

dim1 = 3
dim2 = 3

M1 = tf.Variable(tf.random_uniform([dim1,dim2],minval=-20, maxval=20))
M2 = tf.Variable(tf.random_uniform([dim2,dim1],minval=-20, maxval=20)) 

W_cl = tf.Variable(tf.ones([4*dim1, 1]))

W_rel = tf.Variable(tf.ones([4*dim1, 1]))
W_relcl = tf.Variable(tf.ones([1, 1]))

b_cl = tf.Variable(tf.ones([1]))

M = tf.concat(1,[M1,M2,M1-M2,M1*M2])

logs = tf.matmul(M,W_cl)+ b_cl
soft = tf.nn.softmax(logs)

relu = tf.nn.relu(tf.matmul(M,W_rel)+ b_cl)
logsrelu = tf.matmul(relu,W_relcl)+ b_cl
softrelu = tf.nn.softmax(logsrelu)

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)


##### Print results

results = sess.run([M1,M2,M,W_cl,b_cl,logs,soft,relu,logsrelu,softrelu])
print "Hp\n",results[0]
print "Hh\n",results[1]
print "Concatenated\n",results[2]
print "W\n",results[3].transpose()
print "b\n",results[4]
print "logits\n",results[5]
print "maxsoft\n",results[6]
print "relu\n",results[7]
print "logitssrelu\n",results[8]
print "maxsoftrelu\n",results[9]



Hp
[[ 17.54130554  -8.90909195 -11.56913757]
 [ -6.45233631   6.53700829   1.03490829]
 [ 18.73347473  -2.52202034 -18.44117165]]
Hh
[[  9.42677498 -16.72008896   4.15140533]
 [-19.28181076  -9.3717289   -7.19891548]
 [ -2.93381119  -3.1003418   17.21966553]]
Concatenated
[[  17.54130554   -8.90909195  -11.56913757    9.42677498  -16.72008896
     4.15140533    8.11453056    7.81099701  -15.72054291  165.35794067
   148.96081543  -48.02817917]
 [  -6.45233631    6.53700829    1.03490829  -19.28181076   -9.3717289
    -7.19891548   12.82947445   15.90873718    8.23382378  124.41272736
   -61.26306915   -7.45021725]
 [  18.73347473   -2.52202034  -18.44117165   -2.93381119   -3.1003418
    17.21966553   21.66728592    0.57832146  -35.66083527  -54.96047592
     7.81912518 -317.55081177]]
W
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]
b
[ 1.]
logits
[[ 261.41674805]
 [  58.93860626]
 [-368.15161133]]
maxsoft
[[ 1.]
 [ 1.]
 [ 1.]]
relu
[[ 261.41674805]
 [  58.93860626]
 [   0.      

In [97]:

import tensorflow as tf 

dim1 = 3
W_cl = tf.Variable(tf.zeros([dim1, 1]))

alpha = [0.5,0.5]
pack_tensor = tf.pack(alpha)
alpha = [[0.5,1,1],[0.5,1,1]]
pack_2lists = tf.pack(alpha)
pack_2lists_reshaped = tf.reshape( tf.pack(alpha), [-1] )
print "pack_2lists_reshaped", pack_2lists_reshaped.get_shape()
pack_2lists_reshaped_back = tf.reshape( tf.pack(alpha), [2,3] )

e = [[0.0001,0.001],[0.5,0.001],[0.0003,0.01]]
e = tf.pack(e)
soft_e = tf.nn.softmax(tf.reduce_mean(e,1))
reduce_soft_e = tf.reduce_max(soft_e)

new = tf.concat( 0, [pack_2lists_reshaped,pack_2lists_reshaped] )
new_back = tf.reshape( new , [2,2,3] )

### Session
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

##### Results
results = sess.run( [W_cl, pack_tensor, pack_2lists,pack_2lists_reshaped, e, soft_e, reduce_soft_e, new,
                    pack_2lists_reshaped_back, new_back] , )

print "Hp\n",results[0]
print "pack_tensor", results[1]
print "pack_2lists", results[2]
print "pack_2lists_reshaped", results[3]
print "e ", results[4]
print "soft_e", results[5]
print "reduce_soft_e", results[6]
print "new", results[7]
print "pack_2lists_reshaped_back", results[8]
print "new back", results[9]


pack_2lists_reshaped (6,)
Hp
[[ 0.]
 [ 0.]
 [ 0.]]
pack_tensor [ 0.5  0.5]
pack_2lists [[ 0.5  1.   1. ]
 [ 0.5  1.   1. ]]
pack_2lists_reshaped [ 0.5  1.   1.   0.5  1.   1. ]
e  [[  9.99999975e-05   1.00000005e-03]
 [  5.00000000e-01   1.00000005e-03]
 [  3.00000014e-04   9.99999978e-03]]
soft_e [ 0.30408338  0.39043128  0.3054854 ]
reduce_soft_e 0.390431
new [ 0.5  1.   1.   0.5  1.   1.   0.5  1.   1.   0.5  1.   1. ]
pack_2lists_reshaped_back [[ 0.5  1.   1. ]
 [ 0.5  1.   1. ]]
new back [[[ 0.5  1.   1. ]
  [ 0.5  1.   1. ]]

 [[ 0.5  1.   1. ]
  [ 0.5  1.   1. ]]]
